In [1]:
import pandas as pd
from smtutil import hiveutil, oracle_to_hive
from datetime import datetime
from cx_Oracle import DatabaseError
import numpy as np
import pyspark.sql.functions as F 
import time
from glob import glob
from datetime import timedelta
import os
import shutil
import glob
import sys
sc, spark = hiveutil.get_sc_spark()
hiveutil.register_common_udf(spark)
import tracemalloc
import tempfile
conn = oracle_to_hive.connect_to_Oracle()
import sys
sys.path.insert(1,os.path.realpath("..")+'/src')
from weblog_raw_data_clone import *

Connect to Oracle Successfully
Connect to Oracle Successfully
Connect to Oracle Successfully


### Assign the run date

In [2]:
sdate = '20220911'  # YYYYMMDD
edate = '20220911' # YYYYMMDD

### Assign the path

In [3]:
tmp_dir=tempfile.mkdtemp()
tmp_data_folder_path=os.path.join(os.path.realpath(".."), "data")+tmp_dir

if os.path.exists(tmp_data_folder_path)==False:
    os.mkdir(tmp_data_folder_path+'/')
data_folder_path=os.path.join(os.path.realpath(".."), "data")
print("data_folder_path: "+data_folder_path)
print("tmp_data_folder_path: "+tmp_data_folder_path)

data_folder_path: /app/ws-sunny/git_repo/weblog/legacy/weblog_raw_data_clone/data
tmp_data_folder_path: /app/ws-sunny/git_repo/weblog/legacy/weblog_raw_data_clone/data/tmp/tmp2pzpa61z


### Data Control

In [4]:
loaded_dates = []

In [5]:
# for schedule job
# if os.path.exists(data_folder_path)==True:
#     for date_id in glob(f'{data_folder_path}/date_id=*'):
#         loaded_dates.append(int(date_id[-8:]))
    
#     loaded_dates = [x for x in loaded_dates]
#     print(loaded_dates[-1], today)
# else:
#     print('dest_path not exist')

In [6]:
sdate = datetime.strptime(str(loaded_dates[-1]), '%Y%m%d') if sdate is None else datetime.strptime(str(sdate), '%Y%m%d')
edate = datetime.strptime(str(end_day), '%Y%m%d') if edate is None else datetime.strptime(str(edate), '%Y%m%d')

delta = edate - sdate 
days = []

for i in range(delta.days + 1):
    day = int((sdate + timedelta(days=i)).strftime('%Y%m%d'))
    if day not in set(loaded_dates):
        days.append(day)
print(f"start_date: {sdate}, end_date: {edate}")

start_date: 2022-09-11 00:00:00, end_date: 2022-09-11 00:00:00


In [7]:
# for schedule job
# exclude_near_dates = [] 
# exclude_near_dates.append(int((datetime.now()).strftime("%Y%m%d")))
# exclude_near_dates.append(int((datetime.now() - timedelta(days=1)).strftime("%Y%m%d")))
# exclude_near_dates.append(int((datetime.now() - timedelta(days=2)).strftime("%Y%m%d")))
# days = [x for x in days if x not in exclude_near_dates]

In [8]:
for day in days:
    date_str = datetime.strptime(str(day), '%Y%m%d').strftime('%Y-%m-%d')
    print(day, date_str)

20220911 2022-09-11


### Execute the code

In [9]:
for day in days:
    print(day)
    date_str = datetime.strptime(str(day), '%Y%m%d').strftime('%Y-%m-%d')
    missing_partitions = get_missing_partition(day,data_folder_path)
    print(missing_partitions)
    tracemalloc.start()
    for subr_num_partition in missing_partitions:
        while True:
            try:
                load_ssl(date_str,subr_num_partition,tmp_data_folder_path)
                print(tracemalloc.get_traced_memory())
                break
            except Exception:
                continue

20220911


NameError: name 'os' is not defined